In [1]:
import json
import pandas as pd
import numpy as np

# Useful functions

In [2]:
def load_json(path):
    f = open(path, "r")
    json_loaded = json.load(f)
    f.close()
    return json_loaded

In [3]:
def json_save(obj, name):
    json_content = json.dumps(obj)
    f = open(f"{name}.json","w")
    f.write(json_content)
    f.close()   

# Open required files

In [4]:
domain_warcs = load_json('/project/0_cleaning/output_cleaning/warc_per_dom.json')
domain_scope = pd.read_csv('/project/0_cleaning/output_cleaning/sirens_final.csv')
domain_scope['siren'] = domain_scope['siren'].astype(str)
sirens = np.load('/project/0_cleaning/output_cleaning/sirens.npy')
domain_scope.head(5)

,domain,siren
0,www.easypets.fr,530772565
1,www.a4.fr,388336000
2,www.laptopservice.fr,519216188
3,www.toutespiecesanspermis.fr,382528958
4,www.studiosport.fr,495049132


# Extract data

In [5]:
siren_domains = {}
siren_warcs = {}

for siren in sirens:
    domains = domain_scope.loc[domain_scope['siren']==siren, 'domain'].tolist()
    siren_domains[siren] = domains
    
    warcs_for_siren = []
    for domain in domains:
        warcs = domain_warcs[domain]
        warcs_for_siren += warcs
    siren_warcs[siren] = warcs_for_siren

In [6]:
json_save(siren_domains, 'siren_domains')

In [7]:
len(siren_domains)

3573

# Test data extracted

In [6]:
print(len(domain_warcs['www.easypets.fr']) + len(domain_warcs['www.easyhorses.fr']))
print(len(siren_warcs['530772565']))

5559
5559


In [7]:
print(len(siren_domains))
print(len(siren_warcs))

3573
3573


# Save number of domains as feature

In [8]:
nb_domains = pd.DataFrame()
nb_domains['siren'] = sirens
nb_domains['nb_domains'] = [len(domains) for domains in siren_domains.values()]
nb_domains.to_csv('/project/1_feature_extraction/output_feature_extraction/nb_domains.csv', index=False)
print(nb_domains.shape)
nb_domains.head()

(3573, 2)


,siren,nb_domains
0,332224393,1
1,798102158,1
2,414305508,1
3,533910931,1
4,809944416,1


# Save dictionary of warcs per siren for later use

In [9]:
json_save(siren_warcs, 'siren_warcs')